<a href="https://colab.research.google.com/github/reban87/ML-Projects/blob/main/Task1_NER_Recognition_Transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# `Task 1`: NER using Transformer
prepared by: Rebanta Aryal | 
RPALabs | 31st March, 2022

[Named Entity Recognition(NER)](https://en.wikipedia.org/wiki/Named-entity_recognition) is a subtask of information extraction that seeks to locate and classify named entity mentioned in unstructured text into pre-defined categories such as person names, organizations, locations, medical codes, time expressions, monetary values, percentage etc.

It aims to assign a class to each token ( usually in a single word) in a sequence.Because of this, NER is also referred to as token classification.

##**Simple Transformer**
Simple transformer is built on the top of the excellent Transformers library by [Hugging Face](https://huggingface.co)

In [ ]:
#install simple transformers
!pip install simpletransformers

     |████████████████████████████████| 249 kB 15.5 MB/s 
     |████████████████████████████████| 1.2 MB 52.0 MB/s 
     |████████████████████████████████| 6.6 MB 67.4 MB/s 
     |████████████████████████████████| 325 kB 59.3 MB/s 
     |████████████████████████████████| 3.8 MB 55.1 MB/s 
     |████████████████████████████████| 10.1 MB 50.6 MB/s 
     |████████████████████████████████| 1.7 MB 49.9 MB/s 
     |████████████████████████████████| 43 kB 2.4 MB/s 
     |████████████████████████████████| 596 kB 54.6 MB/s 
     |████████████████████████████████| 895 kB 62.2 MB/s 
     |████████████████████████████████| 67 kB 6.2 MB/s 
     |████████████████████████████████| 144 kB 62.2 MB/s 
     |████████████████████████████████| 181 kB 55.7 MB/s 
     |████████████████████████████████| 63 kB 1.9 MB/s 
     |████████████████████████████████| 1.1 MB 50.6 MB/s 
     |████████████████████████████████| 212 kB 60.6 MB/s 
     |████████████████████████████████| 134 kB 64.0 MB/s 
     |█████████████

In [ ]:
import pandas as pd

In [ ]:
data=pd.read_csv("/content/drive/MyDrive/archive/ner_datasetreference.csv",encoding="latin1") #Latin-1 encodes just the first 256 code points of the Unicode character set,
data.head(10)

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O
5,NaN,through,IN,O
6,NaN,London,NNP,B-geo
7,NaN,to,TO,O
8,NaN,protest,VB,O
9,NaN,the,DT,O


In [ ]:
data.head(30)

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O
5,NaN,through,IN,O
6,NaN,London,NNP,B-geo
7,NaN,to,TO,O
8,NaN,protest,VB,O
9,NaN,the,DT,O


###Let us preprocess the data by refilling the NaN value with  the sentence 1 and so on...

In [ ]:
data=data.fillna(method='ffill') #propagate last valid observation using ffill method
data.head(30)

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,Sentence: 1,of,IN,O
2,Sentence: 1,demonstrators,NNS,O
3,Sentence: 1,have,VBP,O
4,Sentence: 1,marched,VBN,O
5,Sentence: 1,through,IN,O
6,Sentence: 1,London,NNP,B-geo
7,Sentence: 1,to,TO,O
8,Sentence: 1,protest,VB,O
9,Sentence: 1,the,DT,O


In [ ]:
from sklearn.preprocessing import LabelEncoder

from sklearn.metrics import accuracy_score

``Label Encoding`` is being used  to encode the sentence column by random values, as we need to pass the model the encoded format.

In [ ]:
data["Sentence #"]=LabelEncoder().fit_transform(data["Sentence #"])
data.head(30)

,Sentence #,Word,POS,Tag
0,0,Thousands,NNS,O
1,0,of,IN,O
2,0,demonstrators,NNS,O
3,0,have,VBP,O
4,0,marched,VBN,O
5,0,through,IN,O
6,0,London,NNP,B-geo
7,0,to,TO,O
8,0,protest,VB,O
9,0,the,DT,O


Since the simple transformer doesnot take the format that we have in our dataset, therefore, we now rename the format of our dataset as below...

In [ ]:
data.rename(columns={"Sentence #":"sentence_id","Word":"words","Tag":"labels"}, inplace=True)

Let us define independent variables be ``sentence_id`` and ``words`` and the dependent variable be ``labels``

In [ ]:
X=data[["sentence_id","words"]]
y=data["labels"]

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test= train_test_split(X,y,test_size=0.2) 

In [ ]:
#Let's create our train and test data
train_data=pd.DataFrame({"sentence_id":X_train["sentence_id"],"words":X_train["words"],"labels":y_train})
test_data=pd.DataFrame({"sentence_id":X_test["sentence_id"],"words":X_test["words"],"labels":y_test})

In [ ]:
train_data.head(10)

,sentence_id,words,labels
524892,15564,.,O
137680,43824,suspect,O
919823,35585,50,O
676429,23237,say,O
750906,27030,reopen,O
946164,36932,compound,O
634835,21133,the,O
109626,42395,published,O
599926,19349,the,O
347660,6561,",",O


In [ ]:
test_data.head(10)

,sentence_id,words,labels
134853,43682,city,O
292346,3766,is,O
241174,1166,by,O
433121,10857,with,O
682726,23549,'s,O
55454,16846,.,O
641562,21480,He,O
944671,36853,on,O
697843,24329,de,B-org
538432,16267,The,O


### Model Training
lets use simple tranformer library using NERModel, NERArgs for building the model

In [ ]:
!pip install setuptools==59.5.0

     |████████████████████████████████| 952 kB 5.2 MB/s 
  Attempting uninstall: setuptools
    Found existing installation: setuptools 61.3.0
    Uninstalling setuptools-61.3.0:
      Successfully uninstalled setuptools-61.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.0 requires tf-estimator-nightly==2.8.0.dev2021122109, which is not installed.
ipykernel 6.11.0 requires setuptools>=60, but you have setuptools 59.5.0 which is incompatible.
google-colab 1.0.0 requires ipykernel~=4.10, but you have ipykernel 6.11.0 which is incompatible.
google-colab 1.0.0 requires ipython~=5.5.0, but you have ipython 7.32.0 which is incompatible.
google-colab 1.0.0 requires tornado~=5.1.0; python_version >= "3.0", but you have tornado 6.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
!pip install simpletransformers


     |████████████████████████████████| 249 kB 5.3 MB/s 
     |████████████████████████████████| 43 kB 2.1 MB/s 
     |████████████████████████████████| 1.2 MB 42.3 MB/s 
     |████████████████████████████████| 10.1 MB 23.2 MB/s 
     |████████████████████████████████| 3.8 MB 39.0 MB/s 
     |████████████████████████████████| 1.7 MB 33.9 MB/s 
     |████████████████████████████████| 325 kB 51.0 MB/s 
     |████████████████████████████████| 6.6 MB 34.8 MB/s 
     |████████████████████████████████| 596 kB 46.4 MB/s 
     |████████████████████████████████| 895 kB 44.1 MB/s 
     |████████████████████████████████| 67 kB 5.4 MB/s 
     |████████████████████████████████| 144 kB 30.1 MB/s 
     |████████████████████████████████| 181 kB 48.5 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 
     |████████████████████████████████| 136 kB 29.5 MB/s 
     |████████████████████████████████| 212 kB 51.7 MB/s 
     |████████████████████████████████| 1.1 MB 43.8 MB/s 
     |██████████████

In [ ]:
from simpletransformers.ner import NERModel,NERArgs

``Unique labels`` of NER  datasets are converted and stored in a list. 

In [ ]:
label = data["labels"].unique().tolist()
label

['O',
 'B-geo',
 'B-gpe',
 'B-per',
 'I-geo',
 'B-org',
 'I-org',
 'B-tim',
 'B-art',
 'I-art',
 'I-per',
 'I-gpe',
 'I-tim',
 'B-nat',
 'B-eve',
 'I-eve',
 'I-nat']

In [ ]:
args = NERArgs()
args.num_train_epochs = 1
args.learning_rate = 1e-4
args.overwrite_output_dir =True
args.train_batch_size = 32
args.eval_batch_size = 32

In [ ]:
model_NER=NERModel('bert','bert-base-cased',labels=label,args=args,use_cuda=True)

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/416M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForTokenClassification: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cas

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

In [ ]:
model_NER.train_model(train_data,eval_data = test_data,acc=accuracy_score)

  0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/1499 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


(1499, 0.18992250606795405)

In [ ]:
result, model_outputs, preds_list = model_NER.eval_model(test_data)

  0%|          | 0/2 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/1462 [00:00<?, ?it/s]

In [ ]:
result

{'eval_loss': 0.1688306446871357,
 'precision': 0.8304639932815453,
 'recall': 0.759091636014713,
 'f1_score': 0.7931754758284177}

In [ ]:
prediction, model_output = model_NER.predict(["What is the capital city of Nepal"])

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
prediction

[[{'What': 'O'},
  {'is': 'O'},
  {'the': 'O'},
  {'capital': 'O'},
  {'city': 'O'},
  {'of': 'O'},
  {'Nepal': 'B-gpe'}]]

In [ ]:
prediction1, model_output = model_NER.predict(["My name is Rebanta and I live in Kathmandu"])

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
prediction1

[[{'My': 'O'},
  {'name': 'O'},
  {'is': 'O'},
  {'Rebanta': 'B-per'},
  {'and': 'O'},
  {'I': 'O'},
  {'live': 'O'},
  {'in': 'O'},
  {'Kathmandu': 'B-geo'}]]

In [ ]:
#save the model
import pickle


In [ ]:
NER_Task = 'NER_Task1.sav'
pickle.dump(model_NER, open(NER_Task, 'wb'))